# Image Processing SS 20 - Assignment - 02

### Deadline is 6.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.


# Exercise 1 - 10 Points

Implement affine transformation with [bicubic interpolation](https://en.wikipedia.org/wiki/Bicubic_interpolation).
Implement the functions `affine_transformation` and `bicubic_interpolation`. Apply some affine transformation of your choice and smooth the output using your bicubic interpolation.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

The [skimage](http://scikit-image.org/) library comes with multiple useful test images.  Let's start with an image of an astronaut. 

In [ ]:
from skimage.data import astronaut
from skimage.color import rgb2gray

In [ ]:
# We use a gray image. All the algorithms should work with color images too.
img = rgb2gray(astronaut() / 255.)
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
def derive_y(image):
    """Computes the derivative of the image w.r.t the y coordinate"""
    derived_image = np.zeros_like(image)
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            if y + 1 < image.shape[1] and y - 1 > 0:
                derived_image[x,y] = (image[x, y + 1] - image[x, y - 1]) / 2.0
    return derived_image

def derive_x(image):
    """Computes the derivative of the image w.r.t the x coordinate"""
    derived_image = np.zeros_like(image)
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            if x + 1 < image.shape[1] and x - 1 > 0:
                derived_image[x,y] = (image[x + 1, y] - image[x - 1, y]) / 2.0
    return derived_image

In [ ]:
dx_img = derive_x(img)
dy_img = derive_y(img)
dxy_img = derive_x(dy_img)

In [ ]:
plt.figure(figsize=(18, 12))
plt.subplot(131)
plt.imshow(img, cmap='gray')
plt.subplot(132)
plt.imshow(dx_img, cmap='gray')
plt.subplot(133)
plt.imshow(dy_img, cmap='gray')
plt.show()

Here are some sample affine transformations to be used later on

In [ ]:
T_scale = np.array([
    [0.75, 0, 0],
    [0, 0.75, 0],
    [0, 0, 1],
])

In [ ]:
T_affine = np.array([
    [1, 0.3, 0],
    [0, 1, 0],
    [0, 0, 1],
])

In [ ]:
# you can use this function to invert the matricies
np.linalg.inv(T_scale)

In [ ]:
def affine_transformation(img, matrix):
    # your code here
    # apply bicubic interpolation
    trans_image = np.zeros_like(img)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            [k,l]=np.dot(np.linalg.inv(matrix),np.array([x,y,1]))[:2]
            if k+1<img.shape[1]and k-1>0 and l+1<img.shape[1]and l-1>0:
                trans_image[x,y] = bicubic_interpolation(img ,[k,l]) 
    return trans_image

In [ ]:
 #if np.dot(np.linalg.inv(matrix),np.array([x,y,1]))[1] < img.shape[1] and y - 1 > 0:

In [ ]:
def bicubic_interpolation(img, indicies):
        # your code here
        n,m=indicies
        i,j=np.array(indicies).astype(int)
        n=n-i
        m=m-j
        A1=np.array([[1,0,0,0],[0,0,1,0],[-3,3,-2,-1],[2,-2,1,1]])
        A2=np.array([[1,0,-3,2],[0,0,3,-2],[0,1,-2,1],[0,0,-1,1]])
        f=np.array([[img[i,j], img[i,j+1], dy_img[i,j], dy_img[i,j+1]],
                    [img[i+1,j], img[i+1,j+1], dy_img[i+1,j], dy_img[i+1,j+1]],
                    [dx_img[i,j], dx_img[i,j+1], dxy_img[i,j], dxy_img[i,j+1]],
                    [dx_img[i+1,j], dx_img[i+1,j+1], dxy_img[i+1,j], dxy_img[i+1,j+1]]])
        alpha_matrix=np.linalg.multi_dot([A1,f,A2])
        Y=np.array([1,m,m**2,m**3])
        X=np.array([1,n,n**2,n**3])           
        return np.linalg.multi_dot([X,alpha_matrix,Y])
    

In [ ]:
img_scale = affine_transformation(img, T_scale)
img_affine = affine_transformation(img, T_affine)

In [ ]:
plt.imshow(img_scale, cmap='gray')
plt.show()

In [ ]:
plt.imshow(img_affine, cmap='gray')
plt.show()
